This notebook has the **Van Leer type advection scheme** which was modified in *Lin et al. (1994)*, and later used for constructing extened multidimensional scheme in *Lin and Rood (1996)*, which is the core advection scheme of the current version of **GEOS-Chem**. This scheme has two ghost points in each side and zero-gradient boundary condition will be applied.

In [1]:
#### Import packages!!!
#import Pkg; Pkg.add("Plots")
using Plots
#Pkg.add("BenchmarkTools")
using BenchmarkTools
#Pkg.add("Flux")
using Flux
#Pkg.add("CSV")
using CSV
#Pkg.add("DelimitedFiles")
using DelimitedFiles
#Pkg.add("Distributions")
using Distributions

In [2]:
#### Initial condition: Square shape
function ic(Δx::Float32, nx::Int32, s1::Array{Float32})
    length = Δx * nx
    for i in 1:nx
        x = Δx*(i-1)
        if i < nx÷3
            s1[i+2] = 0
        elseif nx÷3 <= i < 2*nx÷3
            s1[i+2] = 1e-7
        elseif i >= 2*nx÷3
            s1[i+2] = 0
        end
    end
end

ic (generic function with 1 method)

In [17]:
#### Initial condition: Dirac-delta
function ic_dd(Δx::Float32, nx::Int32, s1::Array{Float32})
    for i in 1:nx
        if i == nx÷6
            s1[i+2] = 1e-7
        else
            s1[i+2] = 0
        end
    end
end

ic_dd (generic function with 1 method)

In [101]:
#### Initial condition: Gaussian shape
function ic_gd(Δx::Float32, nx::Int32, s1::Array{Float32})
    length = Δx * real(nx)
    for i in 1:nx
        x = Δx*real(i-1)
        μ = (nx-1)/2
        σ = (nx-1)/8
        s1[i+2] = 1e-7*exp(-0.5*((i-μ)/σ)^2)
    end
end

ic_gd (generic function with 1 method)

In [3]:
#### Boundary condition: from bc.f90
function bc(nx::Int32, s1::Array{Float32})
    
    s1[1] = s1[3]
    s1[2] = s1[3]
    s1[nx+3] = s1[nx+2]
    s1[nx+4] = s1[nx+2]
    s1[nx+5] = s1[nx+2]
end

bc (generic function with 1 method)

In [4]:
#### Advection: from advection.f90 - Van Leer Type Scheme: Modified version in L94
function advection(s1::Array{Float32}, s2::Array{Float32}, u1::Array{Float32}, Δt::Float32, Δx::Float32, nx::Int32)

    ϕ1 = zeros(Float32, nx+4)
    ϕ2 = zeros(Float32, nx+4)
    δϕ1 = zeros(Float32, nx+4)
    Δϕ1_avg = zeros(Float32, nx+3)
    Δϕ1_mono = zeros(Float32, nx+3)
    δϕ2 = zeros(Float32, nx+4)
    Δϕ2_avg = zeros(Float32, nx+3)
    Δϕ2_mono = zeros(Float32, nx+3)
    FLUX = zeros(Float32, nx+3)
    U = zeros(Float32, nx+1)
    
    U[1:nx] = u1[1:nx]
    U[nx+1] = U[nx]
    
    for i in 1:nx+4 ## ϕ is defined to mean volume of cell boundary
        ϕ1[i] = (s1[i]+s1[i+1])/2.0
    end
    
    for i in 2:nx+4
        δϕ1[i] = ϕ1[i]-ϕ1[i-1]
    end
    
    for i in 2:nx+3
        Δϕ1_avg[i] = (δϕ1[i]+δϕ1[i+1])/2.0
    end
    
    for i in 2:nx+3 ## Flux limiter
        ϕ1_min = minimum((ϕ1[i-1], ϕ1[i], ϕ1[i+1]))
        ϕ1_max = maximum((ϕ1[i-1], ϕ1[i], ϕ1[i+1]))
        Δϕ1_mono[i] = sign(Δϕ1_avg[i]) * minimum((abs(Δϕ1_avg[i]), 2*(ϕ1[i]-ϕ1_min), 2*(ϕ1_max-ϕ1[i])))
    end
    
    for i in 3:nx+3
        courant = U[i-2]*Δt/Δx
        if abs(courant) > 1
            println("CFL not satisfied")
        end
        if U[i-2] >= 0
            FLUX[i] = U[i-2] * (ϕ1[i-1] + Δϕ1_mono[i-1]*(1-courant)/2.0)
        elseif U[i-2] < 0
            FLUX[i] = U[i-2] * (ϕ1[i] - Δϕ1_mono[i]*(1+courant)/2.0)
        end
    end
    
    for i in 3:nx+2
        ϕ2[i] = ϕ1[i] - Δt/Δx * (FLUX[i+1]-FLUX[i])
    end
    
    ϕ2[2] = 0
    ϕ2[nx+3] = ϕ2[nx+2]
    
    for i in 3:nx+2
        δϕ2[i] = ϕ2[i]-ϕ2[i-1]
    end
    
    for i in 3:nx+2
        s2[i] = ϕ2[i] - 1/2 * δϕ2[i]
        if s2[i] < 0
            s2[i] = 0
        end
    end
    
end

advection (generic function with 1 method)

In [5]:
function update(s1::Array{Float32}, s2::Array{Float32}, nx::Int32)
    s1[3:nx+2] = s2[3:nx+2]
end

update (generic function with 1 method)

In [6]:
#### Programming advection scheme
function pgm(nx::Int32, u::Matrix{Float32}, Δt::Float32, Δx::Float32, nstep::Int32, maxstep::Int32)
    ## Set array
    s1 = zeros(Float32, nx+5)
    s2 = zeros(Float32, nx+5)
    strue = zeros(Float32, nx+5)
    history = zeros(Float32, nx, maxstep)
    
    ## Initialize
    ic(Δx, nx, s1)
    
    ## Initial condition = true solution after single loop
    strue[3:nx+2] = s1[3:nx+2]
    
    ## Integrate
    for n in 1:nstep
        bc(nx, s1)
        advection(s1, s2, u[:,n], Δt, Δx, nx)
        update(s1, s2, nx)
        history[1:nx,n] = s2[3:nx+2]
    end
    
    return history, u
end

pgm (generic function with 1 method)

In [7]:
## To collect computational time - @benchmark can be also used
pgm_time(nx::Int32, u::Matrix{Float32}, Δt::Float32, Δx::Float32, nstep::Int32, maxstep::Int32) = 
    @time pgm(nx, u, Δt, Δx, nstep, maxstep)

pgm_time (generic function with 1 method)

In [ ]:
## Collect velocity data
vel_GEOS_Array = readdlm("U_Jan_NASA_GMAO.csv", ',', Float32)

In [11]:
history = pgm_time(Int32(201), vel_GEOS_Array, Float32(300*1), Float32(27034.3*1), Int32(2880*1/1), Int32(2880*1/1))

  0.036952 seconds (40.33 k allocations: 36.665 MiB)


(Float32[0.0 0.0 … 5.012261f-10 4.9581195f-10; 0.0 0.0 … 1.4422974f-9 1.434089f-9; … ; 0.0 0.0 … 8.185794f-8 8.163407f-8; 0.0 0.0 … 8.228788f-8 8.209224f-8], Float32[-9.115731 -9.115731 … -4.8329253 -4.886759; -9.021981 -9.021981 … -4.106363 -3.4180088; … ; 5.7748933 5.7748933 … 0.9053558 -0.054727674; 5.3530183 5.3530183 … 0.89217216 0.1386317])

In [111]:
scalar_GEOS_Array = history[1]

201×2880 Matrix{Float32}:
 2.16313f-11  1.80199f-11  1.64783f-11  …  4.77569f-10  4.72405f-10
 4.69825f-11  4.40119f-11  4.21784f-11     1.3742f-9    1.36637f-9
 5.49947f-11  5.62629f-11  5.66297f-11     2.18052f-9   2.17955f-9
 6.41227f-11  6.53918f-11  6.66649f-11     2.94825f-9   2.96439f-9
 7.44524f-11  7.56622f-11  7.68281f-11     3.72059f-9   3.74365f-9
 8.61788f-11  8.72166f-11  8.82477f-11  …  4.4725f-9    4.48467f-9
 9.97278f-11  1.00643f-10  1.01587f-10     5.19374f-9   5.20175f-9
 1.1556f-10   1.16505f-10  1.17469f-10     5.98997f-9   6.01007f-9
 1.33756f-10  1.34815f-10  1.35804f-10     6.99454f-9   7.00317f-9
 1.54599f-10  1.55538f-10  1.56364f-10     8.15276f-9   8.15654f-9
 1.77391f-10  1.77852f-10  1.78348f-10  …  9.364f-9     9.39967f-9
 2.03856f-10  2.04059f-10  2.04262f-10     1.06359f-8   1.05954f-8
 2.3511f-10   2.35117f-10  2.34843f-10     1.16703f-8   1.1556f-8
 ⋮                                      ⋱               
 1.52375f-10  1.527f-10    1.53431f-10     4.9

In [112]:
#### Coarse-grainning 1x
vel_GEOS_Array_1x_2x = zeros(Float32, 201, 1440*1)
scalar_GEOS_Array_1x_2x = zeros(Float32, 201, 1440*1)
for j in 1:1440*1
    for i in 1:201
        for k in 0:1
            vel_GEOS_Array_1x_2x[i,j] += vel_GEOS_Array[i,2*j-k]/2
            scalar_GEOS_Array_1x_2x[i,j] += scalar_GEOS_Array[i,2*j-k]/2
        end
    end
end

vel_GEOS_Array_1x_4x = zeros(Float32, 201, 720*1)
scalar_GEOS_Array_1x_4x = zeros(Float32, 201, 720*1)
for j in 1:720*1
    for i in 1:201
        for k in 0:3
            vel_GEOS_Array_1x_4x[i,j] += vel_GEOS_Array[i,4*j-k]/4
            scalar_GEOS_Array_1x_4x[i,j] += scalar_GEOS_Array[i,4*j-k]/4
        end
    end
end

vel_GEOS_Array_1x_8x = zeros(Float32, 201, 360*1)
scalar_GEOS_Array_1x_8x = zeros(Float32, 201, 360*1)
for j in 1:360*1
    for i in 1:201
        for k in 0:7
            vel_GEOS_Array_1x_8x[i,j] += vel_GEOS_Array[i,8*j-k]/8
            scalar_GEOS_Array_1x_8x[i,j] += scalar_GEOS_Array[i,8*j-k]/8
        end
    end
end

vel_GEOS_Array_1x_16x = zeros(Float32, 201, 180*1)
scalar_GEOS_Array_1x_16x = zeros(Float32, 201, 180*1)
for j in 1:180*1
    for i in 1:201
        for k in 0:15
            vel_GEOS_Array_1x_16x[i,j] += vel_GEOS_Array[i,16*j-k]/16
            scalar_GEOS_Array_1x_16x[i,j] += scalar_GEOS_Array[i,16*j-k]/16
        end
    end
end

vel_GEOS_Array_1x_32x = zeros(Float32, 201, 90*1)
scalar_GEOS_Array_1x_32x = zeros(Float32, 201, 90*1)
for j in 1:90*1
    for i in 1:201
        for k in 0:31
            vel_GEOS_Array_1x_32x[i,j] += vel_GEOS_Array[i,32*j-k]/32
            scalar_GEOS_Array_1x_32x[i,j] += scalar_GEOS_Array[i,32*j-k]/32
        end
    end
end

vel_GEOS_Array_1x_64x = zeros(Float32, 201, 45*1)
scalar_GEOS_Array_1x_64x = zeros(Float32, 201, 45*1)
for j in 1:45*1
    for i in 1:201
        for k in 0:63
            vel_GEOS_Array_1x_64x[i,j] += vel_GEOS_Array[i,64*j-k]/64
            scalar_GEOS_Array_1x_64x[i,j] += scalar_GEOS_Array[i,64*j-k]/64
        end
    end
end

In [113]:
#### Coarse-grainning 2x
vel_GEOS_Array_2x = zeros(Float32, 101, 2880*1)
scalar_GEOS_Array_2x = zeros(Float32, 101, 2880*1)
for j in 1:2880*1
    for i in 1:100
        vel_GEOS_Array_2x[i,j] = (vel_GEOS_Array[2*i-1,j] + vel_GEOS_Array[2*i,j])/2
        scalar_GEOS_Array_2x[i,j] = (scalar_GEOS_Array[2*i-1,j] + scalar_GEOS_Array[2*i,j])/2
    end
    vel_GEOS_Array_2x[101,j] = vel_GEOS_Array[201,j]
    scalar_GEOS_Array_2x[101,j] = scalar_GEOS_Array[201,j]
end

vel_GEOS_Array_2x_2x = zeros(Float32, 101, 1440*1)
scalar_GEOS_Array_2x_2x = zeros(Float32, 101, 1440*1)
for j in 1:1440*1
    for i in 1:100
        vel_GEOS_Array_2x_2x[i,j] = (vel_GEOS_Array[2*i-1,2*j-1] + vel_GEOS_Array[2*i,2*j-1] + 
                                    vel_GEOS_Array[2*i-1,2*j] + vel_GEOS_Array[2*i,2*j])/4
        scalar_GEOS_Array_2x_2x[i,j] = (scalar_GEOS_Array[2*i-1,2*j-1] + scalar_GEOS_Array[2*i,2*j-1] + 
                                    scalar_GEOS_Array[2*i-1,2*j] + scalar_GEOS_Array[2*i,2*j])/4
    end
    vel_GEOS_Array_2x_2x[101,j] = (vel_GEOS_Array[201,2*j-1] + vel_GEOS_Array[201,2*j])/2
    scalar_GEOS_Array_2x_2x[101,j] = (scalar_GEOS_Array[201,2*j-1] + scalar_GEOS_Array[201,2*j])/2
end

vel_GEOS_Array_2x_4x = zeros(Float32, 101, 720*1)
scalar_GEOS_Array_2x_4x = zeros(Float32, 101, 720*1)
for j in 1:720*1
    for i in 1:100
        vel_GEOS_Array_2x_4x[i,j] = (vel_GEOS_Array[2*i-1,4*j-3] + vel_GEOS_Array[2*i,4*j-3] + 
                                    vel_GEOS_Array[2*i-1,4*j-2] + vel_GEOS_Array[2*i,4*j-2] + 
                                    vel_GEOS_Array[2*i-1,4*j-1] + vel_GEOS_Array[2*i,4*j-1] + 
                                    vel_GEOS_Array[2*i-1,4*j] + vel_GEOS_Array[2*i,4*j])/8
        scalar_GEOS_Array_2x_4x[i,j] = (scalar_GEOS_Array[2*i-1,4*j-3] + scalar_GEOS_Array[2*i,4*j-3] + 
                                    scalar_GEOS_Array[2*i-1,4*j-2] + scalar_GEOS_Array[2*i,4*j-2] + 
                                    scalar_GEOS_Array[2*i-1,4*j-1] + scalar_GEOS_Array[2*i,4*j-1] + 
                                    scalar_GEOS_Array[2*i-1,4*j] + scalar_GEOS_Array[2*i,4*j])/8
    end
    vel_GEOS_Array_2x_4x[101,j] = (vel_GEOS_Array[201,4*j-3] + vel_GEOS_Array[201,4*j-2] + 
                                    vel_GEOS_Array[201,4*j-1] + vel_GEOS_Array[201,4*j])/4
    scalar_GEOS_Array_2x_4x[101,j] = (scalar_GEOS_Array[201,4*j-3] + scalar_GEOS_Array[201,4*j-2] + 
                                    scalar_GEOS_Array[201,4*j-1] + scalar_GEOS_Array[201,4*j])/4
end

vel_GEOS_Array_2x_8x = zeros(Float32, 101, 360*1)
scalar_GEOS_Array_2x_8x = zeros(Float32, 101, 360*1)
for j in 1:360*1
    for i in 1:100
        for k in 0:7
            for l in 0:1
                vel_GEOS_Array_2x_8x[i,j] += vel_GEOS_Array[2*i-l,8*j-k]/16
                scalar_GEOS_Array_2x_8x[i,j] += scalar_GEOS_Array[2*i-l,8*j-k]/16
            end
        end
    end
    for k in 0:7
        vel_GEOS_Array_2x_8x[101,j] += vel_GEOS_Array[201,8*j-k]/8
        scalar_GEOS_Array_2x_8x[101,j] += scalar_GEOS_Array[201,8*j-k]/8
    end
end

vel_GEOS_Array_2x_16x = zeros(Float32, 101, 180*1)
scalar_GEOS_Array_2x_16x = zeros(Float32, 101, 180*1)
for j in 1:180*1
    for i in 1:100
        for k in 0:15
            for l in 0:1
                vel_GEOS_Array_2x_16x[i,j] += vel_GEOS_Array[2*i-l,16*j-k]/32
                scalar_GEOS_Array_2x_16x[i,j] += scalar_GEOS_Array[2*i-l,16*j-k]/32
            end
        end
    end
    for k in 0:15
        vel_GEOS_Array_2x_16x[101,j] += vel_GEOS_Array[201,16*j-k]/16
        scalar_GEOS_Array_2x_16x[101,j] += scalar_GEOS_Array[201,16*j-k]/16
    end
end

vel_GEOS_Array_2x_32x = zeros(Float32, 101, 90*1)
scalar_GEOS_Array_2x_32x = zeros(Float32, 101, 90*1)
for j in 1:90*1
    for i in 1:100
        for k in 0:31
            for l in 0:1
                vel_GEOS_Array_2x_32x[i,j] += vel_GEOS_Array[2*i-l,32*j-k]/64
                scalar_GEOS_Array_2x_32x[i,j] += scalar_GEOS_Array[2*i-l,32*j-k]/64
            end
        end
    end
    for k in 0:31
        vel_GEOS_Array_2x_32x[101,j] += vel_GEOS_Array[201,32*j-k]/32
        scalar_GEOS_Array_2x_32x[101,j] += scalar_GEOS_Array[201,32*j-k]/32
    end
end

vel_GEOS_Array_2x_64x = zeros(Float32, 101, 45*1)
scalar_GEOS_Array_2x_64x = zeros(Float32, 101, 45*1)
for j in 1:45*1
    for i in 1:100
        for k in 0:63
            for l in 0:1
                vel_GEOS_Array_2x_64x[i,j] += vel_GEOS_Array[2*i-l,64*j-k]/128
                scalar_GEOS_Array_2x_64x[i,j] += scalar_GEOS_Array[2*i-l,64*j-k]/128
            end
        end
    end
    for k in 0:63
        vel_GEOS_Array_2x_64x[101,j] += vel_GEOS_Array[201,64*j-k]/64
        scalar_GEOS_Array_2x_64x[101,j] += scalar_GEOS_Array[201,64*j-k]/64
    end
end

In [114]:
#### Coarse-grainning 4x
vel_GEOS_Array_4x = zeros(Float32, 51, 2880*1)
scalar_GEOS_Array_4x = zeros(Float32, 51, 2880*1)
## Averaging between two grid points to conserve mass
for j in 1:2880*1
    for i in 1:50
        for l in 0:3
            vel_GEOS_Array_4x[i,j] += vel_GEOS_Array[4*i-l,j]/4
            scalar_GEOS_Array_4x[i,j] += scalar_GEOS_Array[4*i-l,j]/4
        end
    end
    vel_GEOS_Array_4x[51,j] += vel_GEOS_Array[201,j]
    scalar_GEOS_Array_4x[51,j] += scalar_GEOS_Array[201,j]
end

vel_GEOS_Array_4x_2x = zeros(Float32, 51, 1440*1)
scalar_GEOS_Array_4x_2x = zeros(Float32, 51, 1440*1)
for j in 1:1440*1
    for i in 1:50
        for k in 0:1
            for l in 0:3
                vel_GEOS_Array_4x_2x[i,j] += vel_GEOS_Array[4*i-l,2*j-k]/8
                scalar_GEOS_Array_4x_2x[i,j] += scalar_GEOS_Array[4*i-l,2*j-k]/8
            end
        end
    end
    for k in 0:1
        vel_GEOS_Array_4x_2x[51,j] += vel_GEOS_Array[201,2*j-k]/2
        scalar_GEOS_Array_4x_2x[51,j] += scalar_GEOS_Array[201,2*j-k]/2
    end
end

vel_GEOS_Array_4x_4x = zeros(Float32, 51, 720*1)
scalar_GEOS_Array_4x_4x = zeros(Float32, 51, 720*1)
for j in 1:720*1
    for i in 1:50
        for k in 0:3
            for l in 0:3
                vel_GEOS_Array_4x_4x[i,j] += vel_GEOS_Array[4*i-l,4*j-k]/16
                scalar_GEOS_Array_4x_4x[i,j] += scalar_GEOS_Array[4*i-l,4*j-k]/16
            end
        end
    end
    for k in 0:3
        vel_GEOS_Array_4x_4x[51,j] += vel_GEOS_Array[201,4*j-k]/4
        scalar_GEOS_Array_4x_4x[51,j] += scalar_GEOS_Array[201,4*j-k]/4
    end
end

vel_GEOS_Array_4x_8x = zeros(Float32, 51, 360*1)
scalar_GEOS_Array_4x_8x = zeros(Float32, 51, 360*1)
for j in 1:360*1
    for i in 1:50
        for k in 0:7
            for l in 0:3
                vel_GEOS_Array_4x_8x[i,j] += vel_GEOS_Array[4*i-l,8*j-k]/32
                scalar_GEOS_Array_4x_8x[i,j] += scalar_GEOS_Array[4*i-l,8*j-k]/32
            end
        end
    end
    for k in 0:7
        vel_GEOS_Array_4x_8x[51,j] += vel_GEOS_Array[201,8*j-k]/8
        scalar_GEOS_Array_4x_8x[51,j] += scalar_GEOS_Array[201,8*j-k]/8
    end
end

vel_GEOS_Array_4x_16x = zeros(Float32, 51, 180*1)
scalar_GEOS_Array_4x_16x = zeros(Float32, 51, 180*1)
for j in 1:180*1
    for i in 1:50
        for k in 0:15
            for l in 0:3
                vel_GEOS_Array_4x_16x[i,j] += vel_GEOS_Array[4*i-l,16*j-k]/64
                scalar_GEOS_Array_4x_16x[i,j] += scalar_GEOS_Array[4*i-l,16*j-k]/64
            end
        end
    end
    for k in 0:15
        vel_GEOS_Array_4x_16x[51,j] += vel_GEOS_Array[201,16*j-k]/16
        scalar_GEOS_Array_4x_16x[51,j] += scalar_GEOS_Array[201,16*j-k]/16
    end
end

vel_GEOS_Array_4x_32x = zeros(Float32, 51, 90*1)
scalar_GEOS_Array_4x_32x = zeros(Float32, 51, 90*1)
for j in 1:90*1
    for i in 1:50
        for k in 0:31
            for l in 0:3
                vel_GEOS_Array_4x_32x[i,j] += vel_GEOS_Array[4*i-l,32*j-k]/128
                scalar_GEOS_Array_4x_32x[i,j] += scalar_GEOS_Array[4*i-l,32*j-k]/128
            end
        end
    end
    for k in 0:31
        vel_GEOS_Array_4x_32x[51,j] += vel_GEOS_Array[201,32*j-k]/32
        scalar_GEOS_Array_4x_32x[51,j] += scalar_GEOS_Array[201,32*j-k]/32
    end
end

vel_GEOS_Array_4x_64x = zeros(Float32, 51, 45*1)
scalar_GEOS_Array_4x_64x = zeros(Float32, 51, 45*1)
for j in 1:45*1
    for i in 1:50
        for k in 0:63
            for l in 0:3
                vel_GEOS_Array_4x_64x[i,j] += vel_GEOS_Array[4*i-l,64*j-k]/256
                scalar_GEOS_Array_4x_64x[i,j] += scalar_GEOS_Array[4*i-l,64*j-k]/256
            end
        end
    end
    for k in 0:63
        vel_GEOS_Array_4x_64x[51,j] += vel_GEOS_Array[201,64*j-k]/64
        scalar_GEOS_Array_4x_64x[51,j] += scalar_GEOS_Array[201,64*j-k]/64
    end
end

In [115]:
#### Coarse-grainning 8x
vel_GEOS_Array_8x = zeros(Float32, 26, 2880*1)
scalar_GEOS_Array_8x = zeros(Float32, 26, 2880*1)
## Averaging between two grid points to conserve mass
for j in 1:2880*1
    for i in 1:25
        for l in 0:7
            vel_GEOS_Array_8x[i,j] += vel_GEOS_Array[8*i-l,j]/8
            scalar_GEOS_Array_8x[i,j] += scalar_GEOS_Array[8*i-l,j]/8
        end
    end
    vel_GEOS_Array_8x[26,j] += vel_GEOS_Array[201,j]
    scalar_GEOS_Array_8x[26,j] += scalar_GEOS_Array[201,j]
end

vel_GEOS_Array_8x_2x = zeros(Float32, 26, 1440*1)
scalar_GEOS_Array_8x_2x = zeros(Float32, 26, 1440*1)
for j in 1:1440*1
    for i in 1:25
        for k in 0:1
            for l in 0:7
                vel_GEOS_Array_8x_2x[i,j] += vel_GEOS_Array[8*i-l,2*j-k]/16
                scalar_GEOS_Array_8x_2x[i,j] += scalar_GEOS_Array[8*i-l,2*j-k]/16
            end
        end
    end
    for k in 0:1
        vel_GEOS_Array_8x_2x[26,j] += vel_GEOS_Array[201,2*j-k]/2
        scalar_GEOS_Array_8x_2x[26,j] += scalar_GEOS_Array[201,2*j-k]/2
    end
end

vel_GEOS_Array_8x_4x = zeros(Float32, 26, 720*1)
scalar_GEOS_Array_8x_4x = zeros(Float32, 26, 720*1)
for j in 1:720*1
    for i in 1:25
        for k in 0:3
            for l in 0:7
                vel_GEOS_Array_8x_4x[i,j] += vel_GEOS_Array[8*i-l,4*j-k]/32
                scalar_GEOS_Array_8x_4x[i,j] += scalar_GEOS_Array[8*i-l,4*j-k]/32
            end
        end
    end
    for k in 0:3
        vel_GEOS_Array_8x_4x[26,j] += vel_GEOS_Array[201,4*j-k]/4
        scalar_GEOS_Array_8x_4x[26,j] += scalar_GEOS_Array[201,4*j-k]/4
    end
end

vel_GEOS_Array_8x_8x = zeros(Float32, 26, 360*1)
scalar_GEOS_Array_8x_8x = zeros(Float32, 26, 360*1)
for j in 1:360*1
    for i in 1:25
        for k in 0:7
            for l in 0:7
                vel_GEOS_Array_8x_8x[i,j] += vel_GEOS_Array[8*i-l,8*j-k]/64
                scalar_GEOS_Array_8x_8x[i,j] += scalar_GEOS_Array[8*i-l,8*j-k]/64
            end
        end
    end
    for k in 0:7
        vel_GEOS_Array_8x_8x[26,j] += vel_GEOS_Array[201,8*j-k]/8
        scalar_GEOS_Array_8x_8x[26,j] += scalar_GEOS_Array[201,8*j-k]/8
    end
end

vel_GEOS_Array_8x_16x = zeros(Float32, 26, 180*1)
scalar_GEOS_Array_8x_16x = zeros(Float32, 26, 180*1)
for j in 1:180*1
    for i in 1:25
        for k in 0:15
            for l in 0:7
                vel_GEOS_Array_8x_16x[i,j] += vel_GEOS_Array[8*i-l,16*j-k]/128
                scalar_GEOS_Array_8x_16x[i,j] += scalar_GEOS_Array[8*i-l,16*j-k]/128
            end
        end
    end
    for k in 0:15
        vel_GEOS_Array_8x_16x[26,j] += vel_GEOS_Array[201,16*j-k]/16
        scalar_GEOS_Array_8x_16x[26,j] += scalar_GEOS_Array[201,16*j-k]/16
    end
end

vel_GEOS_Array_8x_32x = zeros(Float32, 26, 90*1)
scalar_GEOS_Array_8x_32x = zeros(Float32, 26, 90*1)
for j in 1:90*1
    for i in 1:25
        for k in 0:31
            for l in 0:7
                vel_GEOS_Array_8x_32x[i,j] += vel_GEOS_Array[8*i-l,32*j-k]/256
                scalar_GEOS_Array_8x_32x[i,j] += scalar_GEOS_Array[8*i-l,32*j-k]/256
            end
        end
    end
    for k in 0:31
        vel_GEOS_Array_8x_32x[26,j] += vel_GEOS_Array[201,32*j-k]/32
        scalar_GEOS_Array_8x_32x[26,j] += scalar_GEOS_Array[201,32*j-k]/32
    end
end

vel_GEOS_Array_8x_64x = zeros(Float32, 26, 45*1)
scalar_GEOS_Array_8x_64x = zeros(Float32, 26, 45*1)
for j in 1:45*1
    for i in 1:25
        for k in 0:63
            for l in 0:7
                vel_GEOS_Array_8x_64x[i,j] += vel_GEOS_Array[8*i-l,64*j-k]/512
                scalar_GEOS_Array_8x_64x[i,j] += scalar_GEOS_Array[8*i-l,64*j-k]/512
            end
        end
    end
    for k in 0:63
        vel_GEOS_Array_8x_64x[26,j] += vel_GEOS_Array[201,64*j-k]/64
        scalar_GEOS_Array_8x_64x[26,j] += scalar_GEOS_Array[201,64*j-k]/64
    end
end

In [116]:
#### Coarse-grainning 16x
vel_GEOS_Array_16x = zeros(Float32, 13, 2880*1)
scalar_GEOS_Array_16x = zeros(Float32, 13, 2880*1)
## Averaging between two grid points to conserve mass
for j in 1:2880*1
    for i in 1:12
        for l in 0:15
            vel_GEOS_Array_16x[i,j] += vel_GEOS_Array[16*i-l,j]/16
            scalar_GEOS_Array_16x[i,j] += scalar_GEOS_Array[16*i-l,j]/16
        end
    end
    for l in 0:9
        vel_GEOS_Array_16x[13,j] += vel_GEOS_Array[201-l,j]/10
        scalar_GEOS_Array_16x[13,j] += scalar_GEOS_Array[201-l,j]/10
    end
end

vel_GEOS_Array_16x_2x = zeros(Float32, 13, 1440*1)
scalar_GEOS_Array_16x_2x = zeros(Float32, 13, 1440*1)
for j in 1:1440*1
    for i in 1:12
        for k in 0:1
            for l in 0:15
                vel_GEOS_Array_16x_2x[i,j] += vel_GEOS_Array[16*i-l,2*j-k]/32
                scalar_GEOS_Array_16x_2x[i,j] += scalar_GEOS_Array[16*i-l,2*j-k]/32
            end
        end
    end
    for k in 0:1
        for l in 0:9
            vel_GEOS_Array_16x_2x[13,j] += vel_GEOS_Array[201-l,2*j-k]/20
            scalar_GEOS_Array_16x_2x[13,j] += scalar_GEOS_Array[201-l,2*j-k]/20
        end
    end
end

vel_GEOS_Array_16x_4x = zeros(Float32, 13, 720*1)
scalar_GEOS_Array_16x_4x = zeros(Float32, 13, 720*1)
for j in 1:720*1
    for i in 1:12
        for k in 0:3
            for l in 0:15
                vel_GEOS_Array_16x_4x[i,j] += vel_GEOS_Array[16*i-l,4*j-k]/64
                scalar_GEOS_Array_16x_4x[i,j] += scalar_GEOS_Array[16*i-l,4*j-k]/64
            end
        end
    end
    
    for k in 0:3
        for l in 0:9
            vel_GEOS_Array_16x_4x[13,j] += vel_GEOS_Array[201-l,4*j-k]/40
            scalar_GEOS_Array_16x_4x[13,j] += scalar_GEOS_Array[201-l,4*j-k]/40
        end
    end
end

vel_GEOS_Array_16x_8x = zeros(Float32, 13, 360*1)
scalar_GEOS_Array_16x_8x = zeros(Float32, 13, 360*1)
for j in 1:360*1
    for i in 1:12
        for k in 0:7
            for l in 0:15
                vel_GEOS_Array_16x_8x[i,j] += vel_GEOS_Array[16*i-l,8*j-k]/128
                scalar_GEOS_Array_16x_8x[i,j] += scalar_GEOS_Array[16*i-l,8*j-k]/128
            end
        end
    end
    
    for k in 0:7
        for l in 0:9
            vel_GEOS_Array_16x_8x[13,j] += vel_GEOS_Array[201-l,8*j-k]/80
            scalar_GEOS_Array_16x_8x[13,j] += scalar_GEOS_Array[201-l,8*j-k]/80
        end
    end
end

vel_GEOS_Array_16x_16x = zeros(Float32, 13, 180*1)
scalar_GEOS_Array_16x_16x = zeros(Float32, 13, 180*1)
for j in 1:180*1
    for i in 1:12
        for k in 0:15
            for l in 0:15
                vel_GEOS_Array_16x_16x[i,j] += vel_GEOS_Array[16*i-l,16*j-k]/256
                scalar_GEOS_Array_16x_16x[i,j] += scalar_GEOS_Array[16*i-l,16*j-k]/256
            end
        end
    end
    
    for k in 0:15
        for l in 0:9
            vel_GEOS_Array_16x_16x[13,j] += vel_GEOS_Array[201-l,16*j-k]/160
            scalar_GEOS_Array_16x_16x[13,j] += scalar_GEOS_Array[201-l,16*j-k]/160
        end
    end
end

vel_GEOS_Array_16x_32x = zeros(Float32, 13, 90*1)
scalar_GEOS_Array_16x_32x = zeros(Float32, 13, 90*1)
for j in 1:90*1
    for i in 1:12
        for k in 0:31
            for l in 0:15
                vel_GEOS_Array_16x_32x[i,j] += vel_GEOS_Array[16*i-l,32*j-k]/512
                scalar_GEOS_Array_16x_32x[i,j] += scalar_GEOS_Array[16*i-l,32*j-k]/512
            end
        end
    end
    
    for k in 0:31
        for l in 0:9
            vel_GEOS_Array_16x_32x[13,j] += vel_GEOS_Array[201-l,32*j-k]/320
            scalar_GEOS_Array_16x_32x[13,j] += scalar_GEOS_Array[201-l,32*j-k]/320
        end
    end
end

vel_GEOS_Array_16x_64x = zeros(Float32, 13, 45*1)
scalar_GEOS_Array_16x_64x = zeros(Float32, 13, 45*1)
for j in 1:45*1
    for i in 1:12
        for k in 0:63
            for l in 0:15
                vel_GEOS_Array_16x_64x[i,j] += vel_GEOS_Array[16*i-l,64*j-k]/1024
                scalar_GEOS_Array_16x_64x[i,j] += scalar_GEOS_Array[16*i-l,64*j-k]/1024
            end
        end
    end
    
    for k in 0:63
        for l in 0:9
            vel_GEOS_Array_16x_64x[13,j] += vel_GEOS_Array[201-l,64*j-k]/640
            scalar_GEOS_Array_16x_64x[13,j] += scalar_GEOS_Array[201-l,64*j-k]/640
        end
    end
end